<a href="https://colab.research.google.com/github/Ainsworth03/Thriftify-Capstone/blob/main/Capstone_Usage_and_Type_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets

In [2]:
!pip install tensorflow

In [3]:
import pandas as pd
import json
import glob
import re
import os
import shutil
import numpy as np
from PIL import Image, ImageOps
import tensorflow as tf

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"febriannashrullah","key":"42985255d9adebd39782e5474d7c166a"}'}

In [5]:
cwd = os.getcwd()
print(cwd)
!mkdir -p ./kaggle
!cp ./kaggle.json ./kaggle/
!chmod 600 ./kaggle/kaggle.json

/content


In [6]:
import opendatasets

opendatasets.download("https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small")

Dataset URL: https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small


100%|██████████| 565M/565M [00:26<00:00, 22.2MB/s]


In [7]:
file_path = "./fashion-product-images-small/styles.csv"

In [8]:
df = pd.read_csv(file_path, on_bad_lines='skip')

df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [9]:
df_filtered = df[['id', 'articleType', 'baseColour', 'usage']]

df_filtered.head()

,id,articleType,baseColour,usage
0,15970,Shirts,Navy Blue,Casual
1,39386,Jeans,Blue,Casual
2,59263,Watches,Silver,Casual
3,21379,Track Pants,Black,Casual
4,53759,Tshirts,Grey,Casual


In [10]:
df_UsageType = df_filtered[df_filtered['usage'].notna()]
df_UsageType = df_UsageType[df_filtered['articleType'].notna()]
df_UsageType = df_UsageType[['id', 'usage', 'articleType']]
df_UsageType.head()

<ipython-input-10-96de343075fb>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_UsageType = df_UsageType[df_filtered['articleType'].notna()]


,id,usage,articleType
0,15970,Casual,Shirts
1,39386,Casual,Jeans
2,59263,Casual,Watches
3,21379,Casual,Track Pants
4,53759,Casual,Tshirts


In [11]:
df_UsageType.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44107 entries, 0 to 44423
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           44107 non-null  int64 
 1   usage        44107 non-null  object
 2   articleType  44107 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


In [12]:
type_df = df_UsageType['articleType'].value_counts()

type_filtered = type_df[type_df < 100].index
df_UsageType = df_UsageType[~df_UsageType['articleType'].isin(type_filtered)]

df_UsageType['articleType'].value_counts()

,count
articleType,
Tshirts,7067
Shirts,3214
Casual Shoes,2845
Watches,2542
Sports Shoes,2036
Kurtas,1844
Tops,1762
Handbags,1759
Heels,1323


In [13]:
type_df = df_UsageType['usage'].value_counts()

type_filtered = type_df[type_df < 100].index
df_UsageType = df_UsageType[~df_UsageType['usage'].isin(type_filtered)]

df_UsageType['usage'].value_counts()

,count
usage,
Casual,32713
Sports,3929
Ethnic,2910
Formal,2220


In [14]:
df_UsageType.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41772 entries, 0 to 44423
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           41772 non-null  int64 
 1   usage        41772 non-null  object
 2   articleType  41772 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


In [15]:
categories_to_use = ['Tshirts', 'Shirts', 'Casual Shoes', 'Sports Shoes', 'Tops', 'Flip Flops',
    'Sandals', 'Backpacks', 'Formal Shoes', 'Jeans', '  Shorts', 'Trousers',
    'Flats', 'Heels', 'Dresses', 'Skirts', 'Caps', 'Jackets', 'Sweaters', 'Sweatshirts']
## kostumisasi sendiri

df_choosenCategories = df_UsageType[df_UsageType['articleType'].isin(categories_to_use)]

df_choosenCategories['articleType'].value_counts()

,count
articleType,
Tshirts,7067
Shirts,3194
Casual Shoes,2838
Sports Shoes,2036
Tops,1761
Heels,1320
Flip Flops,914
Sandals,894
Backpacks,713


In [16]:
df_choosenCategories = df_choosenCategories.reset_index(drop = True)
df_choosenCategories = df_choosenCategories[['id', 'usage', 'articleType']]
df_choosenCategories.head()

,id,usage,articleType
0,15970,Casual,Shirts
1,39386,Casual,Jeans
2,53759,Casual,Tshirts
3,1855,Casual,Tshirts
4,30805,Ethnic,Shirts


In [17]:
data_revision = [
    {"id": 11906, "articleType": "Flip Flops", "usage": "Casual"},
    {"id": 57491, "articleType": "Formal Shoes", "usage": "Formal"},
    {"id": 59161, "articleType": "Formal Shoes", "usage": "Formal"},
    {"id": 40315, "articleType": "Sports Shoes", "usage": "Sports"},
    {"id": 46291, "articleType": "Sports Shoes", "usage": "Sports"},
    {"id": 34815, "articleType": "Sports Shoes", "usage": "Sports"},
    {"id": 14398, "articleType": "Heels", "usage": "Formal"},
    {"id": 57495, "articleType": "Formal Shoes", "usage": "Formal"},
    {"id": 57490, "articleType": "Formal Shoes", "usage": "Formal"},
    {"id": 35053, "articleType": "Formal Shoes", "usage": "Formal"},
    {"id": 43511, "articleType": "Dresses", "usage": "Casual"},
    {"id": 59994, "articleType": "Dresses", "usage": "Casual"},
    {"id": 34069, "articleType": "Formal Shoes", "usage": "Formal"},
    {"id": 57116, "articleType": "Tops", "usage": "Casual"},
    {"id": 4340, "articleType": "Sports Shoes", "usage": "Sports"},
    {"id": 26587, "articleType": "Sports Shoes", "usage": "Sports"},
    {"id": 34652, "articleType": "Casual Shoes", "usage": "Casual"},
    {"id": 10640, "articleType": "Casual Shoes", "usage": "Casual"},
]

df_rev = pd.DataFrame(data_revision)

# Update nilai kolom tertentu tanpa duplikat
df_choosenCategories.set_index("id", inplace=True)  # Set "id" sebagai index
df_rev.set_index("id", inplace=True)  # Set "id" sebagai index
df_choosenCategories.update(df_rev)  # Perbarui kolom berdasarkan id

# Reset index untuk kembali ke format awal
df_choosenCategories.reset_index(inplace=True)

In [18]:
df_choosenCategories.head()

,id,usage,articleType
0,15970,Casual,Shirts
1,39386,Casual,Jeans
2,53759,Casual,Tshirts
3,1855,Casual,Tshirts
4,30805,Ethnic,Shirts


In [19]:
rules_rev = [
    {"articleType": "Sandals", "usage": "Formal", "new_usage": "Casual"},
    {"articleType": "Flats", "usage": "Sports", "new_usage": "Casual"},
    {"articleType": "Flip Flops", "usage": "Sports", "new_usage": "Casual"},
    {"articleType": "Skirts", "usage": "Sports", "action": "drop"},
    {"articleType": "Sweaters", "usage": "Sports", "new_usage": "Casual"},
]

# Terapkan revisi
for rule in rules_rev:
    if "action" in rule and rule["action"] == "drop":
        # Hapus baris berdasarkan kondisi
        df_choosenCategories = df_choosenCategories[~((df_choosenCategories["articleType"] == rule["articleType"]) & (df_choosenCategories["usage"] == rule["usage"]))]
    else:
        # Perbarui nilai kolom "usage"
        df_choosenCategories.loc[
            (df_choosenCategories["articleType"] == rule["articleType"]) & (df_choosenCategories["usage"] == rule["usage"]),
            "usage",
        ] = rule["new_usage"]

In [20]:
df_choosenCategories['id'] = df_choosenCategories['id'].astype(str) + ".jpg"

article_encoder = LabelEncoder()
usage_encoder = LabelEncoder()

df_choosenCategories['articleType_encoded'] = article_encoder.fit_transform(df_choosenCategories['articleType'])
df_choosenCategories['usage_encoded'] = usage_encoder.fit_transform(df_choosenCategories['usage'])



In [22]:
df_choosenCategories.tail()

,id,usage,articleType,articleType_encoded,usage_encoded
24671,12544.jpg,Casual,Tshirts,18,0
24672,42234.jpg,Casual,Tops,16,0
24673,17036.jpg,Casual,Casual Shoes,2,0
24674,6461.jpg,Casual,Flip Flops,5,0
24675,18842.jpg,Casual,Tshirts,18,0


In [38]:
i = df_choosenCategories[((df_choosenCategories.id == '39401.jpg'))].index
j = df_choosenCategories[((df_choosenCategories.id == '39425.jpg'))].index
df_choosenCategories.drop(i)
df_choosenCategories.drop(j)

,id,usage,articleType,articleType_encoded,usage_encoded
0,15970.jpg,Casual,Shirts,11,0
1,39386.jpg,Casual,Jeans,9,0
2,53759.jpg,Casual,Tshirts,18,0
3,1855.jpg,Casual,Tshirts,18,0
4,30805.jpg,Ethnic,Shirts,11,1
...,...,...,...,...,...
24671,12544.jpg,Casual,Tshirts,18,0
24672,42234.jpg,Casual,Tops,16,0
24673,17036.jpg,Casual,Casual Shoes,2,0
24674,6461.jpg,Casual,Flip Flops,5,0


In [39]:
train_df, temp_df = train_test_split(df_choosenCategories, test_size=0.3, stratify=df_choosenCategories['articleType'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['articleType'], random_state=42)

In [40]:
train_df.head()

,id,usage,articleType,articleType_encoded,usage_encoded
4295,22481.jpg,Casual,Tshirts,18,0
4982,33100.jpg,Casual,Heels,7,0
14721,14066.jpg,Casual,Tshirts,18,0
19709,12668.jpg,Casual,Shirts,11,0
2135,37679.jpg,Casual,Backpacks,0,0


In [27]:
# # Path source direktori gambar dan direktori tujuan untuk gambar terfilter
# source_dir = './fashion-product-images-small/images'  # Gambar-gambar yang ada
# target_dir = './data'  # Direktori baru untuk menyimpan gambar yang sesuai

# # Membuat direktori tujuan jika belum ada
# if not os.path.exists(target_dir):
#     os.makedirs(target_dir)

# # Menyalin gambar-gambar yang sesuai dengan ID dalam DataFrame
# for index, row in df_choosenCategories.iterrows():
#     image_filename = row['id']

#     # Cek apakah file gambar ada di source_dir
#     source_image_path = os.path.join(source_dir, image_filename)
#     if os.path.exists(source_image_path):
#         # Tentukan path tujuan untuk menyalin gambar
#         target_image_path = os.path.join(target_dir, image_filename)

#         # Menyalin gambar ke direktori baru
#         shutil.copy(source_image_path, target_image_path)
#         print(f"image {source_image_path} copied to {target_image_path}")
#     else:
#         df_choosenCategories = df_choosenCategories.drop(index)
#         print(f"File {image_filename} tidak ditemukan di {source_dir}")


Streaming output truncated to the last 5000 lines.
image ./fashion-product-images-small/images/13118.jpg copied to ./data/13118.jpg
image ./fashion-product-images-small/images/13924.jpg copied to ./data/13924.jpg
image ./fashion-product-images-small/images/14759.jpg copied to ./data/14759.jpg
image ./fashion-product-images-small/images/33947.jpg copied to ./data/33947.jpg
image ./fashion-product-images-small/images/38593.jpg copied to ./data/38593.jpg
image ./fashion-product-images-small/images/44815.jpg copied to ./data/44815.jpg
image ./fashion-product-images-small/images/18194.jpg copied to ./data/18194.jpg
image ./fashion-product-images-small/images/8411.jpg copied to ./data/8411.jpg
image ./fashion-product-images-small/images/16832.jpg copied to ./data/16832.jpg
image ./fashion-product-images-small/images/19046.jpg copied to ./data/19046.jpg
image ./fashion-product-images-small/images/7459.jpg copied to ./data/7459.jpg
image ./fashion-product-images-small/images/2319.jpg copied to

In [26]:
# image_dir = target_dir

# # Fungsi untuk memproses file gambar
# def parse_image(filename, article_label, usage_label):
#     # Load gambar dari file
#     image = tf.io.read_file(filename)
#     image = tf.image.decode_jpeg(image, channels=3)
#     image = tf.image.resize(image, [224, 224])  # Resize gambar ke ukuran target
#     image = image / 255.0  # Normalisasi pixel
#     return image, {"article_output": article_label, "usage_output": usage_label}

# # Fungsi untuk augmentasi data
# def augment_image(image, labels):
#     image = tf.image.random_flip_left_right(image)  # Flip horizontal
#     image = tf.image.random_brightness(image, max_delta=0.2)  # Brightness
#     image = tf.image.random_contrast(image, lower=0.8, upper=1.2)  # Kontras
#     return image, labels

# # Fungsi untuk membuat dataset
# def create_dataset(df, image_dir, batch_size=32, is_training=True):
#     # Gabungkan path gambar dengan direktori
#     file_paths = df_choosenCategories['id'].apply(lambda x: f"{image_dir}/{x}").values
#     article_labels = df_choosenCategories['articleType_encoded'].values
#     usage_labels = df_choosenCategories['usage_encoded'].values

#     # Buat dataset dari tensor
#     dataset = tf.data.Dataset.from_tensor_slices((file_paths, article_labels, usage_labels))

#     # Parsing gambar
#     dataset = dataset.map(parse_image, num_parallel_calls=tf.data.AUTOTUNE)

#     # Augmentasi jika training
#     if is_training:
#         dataset = dataset.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
#         dataset = dataset.shuffle(buffer_size=len(df_choosenCategories))

#     # Batch dan prefetch
#     dataset = dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
#     return dataset

# batch_size = 8

# train_dataset = create_dataset(train_df, image_dir, batch_size=batch_size, is_training=True)
# val_dataset = create_dataset(val_df, image_dir, batch_size=batch_size, is_training=False)
# test_dataset = create_dataset(test_df, image_dir, batch_size=batch_size, is_training=False)

In [27]:
#type(train_dataset)

tensorflow.python.data.ops.prefetch_op._PrefetchDataset

In [41]:
df_choosenCategories.head()

,id,usage,articleType,articleType_encoded,usage_encoded
0,15970.jpg,Casual,Shirts,11,0
1,39386.jpg,Casual,Jeans,9,0
2,53759.jpg,Casual,Tshirts,18,0
3,1855.jpg,Casual,Tshirts,18,0
4,30805.jpg,Ethnic,Shirts,11,1


In [42]:
import os
import shutil
import tensorflow as tf
import pandas as pd

# Path source direktori gambar dan direktori tujuan untuk gambar terfilter
source_dir = './fashion-product-images-small/images'  # Gambar-gambar yang ada
target_dir = './data'  # Direktori baru untuk menyimpan gambar yang sesuai

# Membuat direktori tujuan jika belum ada
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Menyalin gambar yang sesuai dengan ID dalam DataFrame
for index, row in df_choosenCategories.iterrows():
    image_filename = row['id']

    # Cek apakah file gambar ada di source_dir
    source_image_path = os.path.join(source_dir, image_filename)
    if os.path.exists(source_image_path):
        # Tentukan path tujuan untuk menyalin gambar
        target_image_path = os.path.join(target_dir, image_filename)

        # Menyalin gambar ke direktori baru
        shutil.copy(source_image_path, target_image_path)
    else:
        # Drop baris dari DataFrame jika file tidak ditemukan
        df_filtered_final = df_filtered_final.drop(index)
        print(f"File {image_filename} tidak ditemukan di {source_dir}")

# Path direktori gambar terfilter
image_dir = target_dir

# Fungsi untuk memproses file gambar
def parse_image(filename, article_label, usage_label):
    # Lazy loading gambar
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])  # Resize gambar ke ukuran target
    image = image / 255.0  # Normalisasi pixel
    return image, {"article_output": article_label, "usage_output": usage_label}

# Fungsi untuk augmentasi data
def augment_image(image, labels):
    image = tf.image.random_flip_left_right(image)  # Flip horizontal
    image = tf.image.random_brightness(image, max_delta=0.2)  # Brightness
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)  # Kontras
    return image, labels

# Generator untuk lazy loading data
def generator(df, image_dir):
    for _, row in df.iterrows():
        file_path = os.path.join(image_dir, row['id'])
        yield file_path, row['articleType_encoded'], row['usage_encoded']

# Fungsi untuk membuat dataset dengan lazy loading
def create_dataset_lazy(df, image_dir, batch_size=8, is_training=True):
    # Dataset menggunakan generator
    dataset = tf.data.Dataset.from_generator(
        lambda: generator(df, image_dir),
        output_signature=(
            tf.TensorSpec(shape=(), dtype=tf.string),  # Path gambar
            tf.TensorSpec(shape=(), dtype=tf.int64),   # Label artikel
            tf.TensorSpec(shape=(), dtype=tf.int64)    # Label penggunaan
        )
    )

    # Parsing gambar
    dataset = dataset.map(parse_image, num_parallel_calls=tf.data.AUTOTUNE)

    # Augmentasi jika training
    if is_training:
        dataset = dataset.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.shuffle(buffer_size=1000)

    # Batch dan prefetch
    dataset = dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

# Membuat dataset dengan lazy loading
train_dataset = create_dataset_lazy(train_df, image_dir, batch_size=8, is_training=True)
val_dataset = create_dataset_lazy(val_df, image_dir, batch_size=8, is_training=False)
test_dataset = create_dataset_lazy(test_df, image_dir, batch_size=8, is_training=False)

In [43]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

# Membuat strategi distribusi untuk multi-GPU
strategy = tf.distribute.MirroredStrategy()

print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

# Menggunakan strategy.scope() untuk membungkus pembuatan dan kompilasi model
with strategy.scope():
    # Load MobileNetV2 sebagai backbone
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    for layer in base_model.layers:
        layer.trainable = False

    for layer in base_model.layers[-10:]:
        layer.trainable = True

    # Tambahkan lapisan pooling global
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu', name='dense_1')(x)
    x = Dense(128, activation='relu', name='dense_2')(x)
    x = Dense(64, activation='relu', name='dense_3')(x)
    x = Dense(32, activation='relu', name='dense_4')(x)

    # Output untuk articleType
    article_output = Dense(len(article_encoder.classes_), activation='softmax', name='article_output')(x)

    # Output untuk usage
    usage_output = Dense(len(usage_encoder.classes_), activation='softmax', name='usage_output')(x)

    # Gabungkan menjadi model akhir
    model = Model(inputs=base_model.input, outputs=[article_output, usage_output])

    # Compile model
    model.compile(
        optimizer='adam',
        loss={
            'article_output': 'sparse_categorical_crossentropy',
            'usage_output': 'sparse_categorical_crossentropy'
        },
        metrics={
            'article_output': 'accuracy',
            'usage_output': 'accuracy'
        }
    )

# Callback untuk mengurangi learning rate
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',       # Pantau nilai val_loss
    factor=0.2,               # Kurangi learning rate menjadi 10% dari nilai saat ini
    patience=3,               # Tunggu 3 epoch tanpa perbaikan sebelum menurunkan lr
    min_lr=1e-6               # Batasi learning rate minimum agar tidak terlalu kecil
)

# Melatih model dengan dataset yang dibagi
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=25,
    callbacks=[reduce_lr]  # Menambahkan callback ke dalam pelatihan
)

Number of devices: 1
Epoch 1/25
   2159/Unknown 87s 37ms/step - article_output_accuracy: 0.5938 - article_output_loss: 1.2389 - loss: 1.7509 - usage_output_accuracy: 0.8207 - usage_output_loss: 0.5120

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


2159/2159 ━━━━━━━━━━━━━━━━━━━━ 98s 42ms/step - article_output_accuracy: 0.5939 - article_output_loss: 1.2388 - loss: 1.7507 - usage_output_accuracy: 0.8208 - usage_output_loss: 0.5120 - val_article_output_accuracy: 0.6903 - val_article_output_loss: 1.1048 - val_loss: 1.5720 - val_usage_output_accuracy: 0.8759 - val_usage_output_loss: 0.4643 - learning_rate: 0.0010
Epoch 2/25
2159/2159 ━━━━━━━━━━━━━━━━━━━━ 91s 41ms/step - article_output_accuracy: 0.7662 - article_output_loss: 0.6524 - loss: 1.0071 - usage_output_accuracy: 0.8715 - usage_output_loss: 0.3547 - val_article_output_accuracy: 0.7624 - val_article_output_loss: 0.8453 - val_loss: 1.3032 - val_usage_output_accuracy: 0.8654 - val_usage_output_loss: 0.4550 - learning_rate: 0.0010
Epoch 3/25
2159/2159 ━━━━━━━━━━━━━━━━━━━━ 89s 40ms/step - article_output_accuracy: 0.7867 - article_output_loss: 0.5843 - loss: 0.9141 - usage_output_accuracy: 0.8753 - usage_output_loss: 0.3299 - val_article_output_accuracy: 0.8111 - val_article_output_l

In [44]:
loss= model.evaluate(test_dataset)
print(f"Test Loss: {loss}")

463/463 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - article_output_accuracy: 0.8696 - article_output_loss: 0.4304 - loss: 0.6884 - usage_output_accuracy: 0.9150 - usage_output_loss: 0.2580
Test Loss: [0.6819921731948853, 0.42863214015960693, 0.25144174695014954, 0.8670629262924194, 0.9162388443946838]


In [45]:
model.save('./result/hybridModel_01.h5')

In [46]:
model.save('./result/hybridModel_01.keras')

## Add Layer